First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [5]:
btw = pd.read_csv(f'{path}/data/btw.csv', converters={'AGS': str})
btw.head()

,Unnamed: 0,AGS,year,post_2013,post_2017,Land,Erststimme,Zweitstimme,Wahlberechtigte,Wähler,Union,SPD,FDP,Linke,Grüne,Andere
0,0,01001000,2002,0,0,1,1.0,NaN,65740,74.618193,33.469238,53.632731,4.154605,1.547274,5.775268,0.395483
1,0,01001000,2002,0,0,1,NaN,1.0,65740,74.618193,29.277939,47.694378,6.980063,1.751131,11.326293,1.948873
2,1,01001000,2005,0,0,1,1.0,NaN,66970,73.170076,34.598588,49.510224,3.267214,4.718175,5.569160,0.789764
3,1,01001000,2005,0,0,1,NaN,1.0,66970,73.170076,28.386596,42.855394,8.562916,6.256887,10.260806,2.112159
4,2,01001000,2009,0,0,1,1.0,NaN,69075,67.438292,29.751197,35.532276,9.129940,10.083078,11.849816,1.069060


Merge with treatment dataset

In [ ]:
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'AGS': str})
#data = btw.join(treatment.set_index(['AGS']))
data = pd.merge(btw, treatment, on='AGS', how='left', indicator=True)
data.head(10)

,AGS,year,post_2013,post_2017,Land,Erststimme,Zweitstimme,Wahlberechtigte,Wähler,Union,...,Grüne,Andere,GEN,treated_0,treated_10,treated_20,treated_30,treated_50,treated_100,_merge
0,01001000,2002,0,0,01,1.0,NaN,65740,74.618193,33.469238,...,5.775268,0.395483,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
1,01001000,2002,0,0,01,NaN,1.0,65740,74.618193,29.277939,...,11.326293,1.948873,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
2,01001000,2005,0,0,01,1.0,NaN,66970,73.170076,34.598588,...,5.569160,5.507938,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
3,01001000,2005,0,0,01,NaN,1.0,66970,73.170076,28.386596,...,10.260806,2.112159,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
4,01001000,2009,0,0,01,1.0,NaN,69075,67.438292,29.751197,...,11.849816,11.152137,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
5,01001000,2009,0,0,01,NaN,1.0,69075,67.438292,24.124681,...,15.720327,5.010412,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
6,01001000,2013,1,0,01,1.0,NaN,70321,66.590350,33.414483,...,9.680313,13.530655,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
7,01001000,2013,1,0,01,NaN,1.0,70321,66.590350,30.187712,...,11.760309,9.184872,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
8,01001000,2017,1,1,01,1.0,NaN,70105,70.885101,30.466455,...,11.989375,19.147181,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
9,01001000,2017,1,1,01,NaN,1.0,70105,70.885101,26.928804,...,14.923331,11.186461,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both


Inspect merge results

In [ ]:
print(data['_merge'].describe())
#not_merged = data[data['_merge'] == 'both']
#not_merged['merge'] = True
not_merged = data[data['_merge'] != 'both']
not_merged['merge'] = False
not_merged = not_merged.reindex(columns = ['AGS'])
not_merged = not_merged.groupby(['AGS'], as_index=True).first()
btw_es['AGS'] = btw_es['Land'] + btw_es['Regierungsbezirk'] + btw_es['Kreis'] + btw_es['Gemeinde']
not_merged['Land'] = not_merged['AGS'].str.slice(start=3, stop=10)

not_merged.to_csv(f'{path}/data/not_merged.csv')

Clean data

In [ ]:
data = data.set_index(['AGS', 'year'])
data = data.drop(columns=['_merge'])
data = data.reindex(columns=['post_2013', 'post_2017', 'GEN', 'Land', 'Wahlberechtigte', 'Wähler', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'Erststimme', 'Zweitstimme', 'treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100'])
#data = data.sort_values(by=['AGS', 'year'])
data.head(10)

Interact treatment dummies with post-treatment indicators

In [ ]:
dummies = ['treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100']
for dummy in dummies:
    data.insert(loc=len(data.columns), column=f'{dummy}*post_2013', value=(data[dummy]*data['post_2013']))
    data.insert(loc=len(data.columns), column=f'{dummy}*post_2017', value=(data[dummy]*data['post_2017']))
data.head(10)

Drop missing values

In [ ]:
data = data.dropna(axis='rows', how='any', subset=['GEN', 'Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'])

Export dataset

In [ ]:
data.to_csv(f'{path}/data/btw_treatment.csv', encoding = 'utf-8-sig')